In [1]:
import xarray as xr
from distributed import Client

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61280,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:61292,Total threads: 2
Dashboard: http://127.0.0.1:61293/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:61283,


In [5]:
store1 = f's3://carbonplan-data-viewer/demo/ScenarioMIP.CCCma.CanESM5.ssp245.r1i1p1f1.day.GARD-SV.tasmax.zarr'
store2 = (
    f's3://carbonplan-data-viewer/demo/ScenarioMIP.CCCma.CanESM5.ssp245.r1i1p1f1.day.tasmax.zarr'
)

input1 = '/Users/andersy005/data/ScenarioMIP.CCCma.CanESM5.ssp245.r1i1p1f1.day.GARD-SV.tasmax.zarr/'
input2 = '/Users/andersy005/data/v20190429/'

chunks = {'time': 30, 'lat': 128, 'lon': 128}
ds1 = xr.open_dataset(input1, engine='zarr', chunks=chunks).isel(time=range(300))
ds2 = xr.open_dataset(input2, engine='zarr', chunks=chunks).isel(time=range(300))

for ds, output in [(ds1, store1), (ds2, store2)]:
    del ds.tasmax.encoding['chunks']
    del ds.tasmax.encoding['preferred_chunks']
    ds.to_zarr(output, consolidated=True, mode='w')

/Users/andersy005/mambaforge/envs/playground/lib/python3.10/site-packages/xarray/core/dataset.py:255: UserWarning: The specified Dask chunks separate the stored chunks along dimension "time" starting at index 30. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/Users/andersy005/mambaforge/envs/playground/lib/python3.10/site-packages/xarray/core/dataset.py:255: UserWarning: The specified Dask chunks separate the stored chunks along dimension "lat" starting at index 128. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/Users/andersy005/mambaforge/envs/playground/lib/python3.10/site-packages/xarray/core/dataset.py:255: UserWarning: The specified Dask chunks separate the stored chunks along dimension "lon" starting at index 128. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/Users/andersy005/mambaforge/envs/playground/lib/python3.10/site-packages/xarray/c

In [6]:
dsets = []
for store in [store1, store2]:
    ds = xr.open_zarr(store)
    dsets.append(ds)

<xarray.Dataset>
Dimensions:  (lat: 721, lon: 1440, time: 300)
Coordinates:
  * lat      (lat) float32 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * lon      (lon) float32 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * time     (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-10-27
Data variables:
    tasmax   (time, lat, lon) float32 dask.array<chunksize=(30, 128, 128), meta=np.ndarray>
Attributes: (12/17)
    Conventions:                     CF-1.8
    activity_id:                     ScenarioMIP
    cmip6_downscaling_contact:       hello@carbonplan.org
    cmip6_downscaling_explainer:     https://carbonplan.org/research/cmip6-do...
    cmip6_downscaling_institution:   CarbonPlan
    cmip6_downscaling_license:       CC-BY-4.0
    ...                              ...
    institution_id:                  CCCma
    member_id:                       r1i1p1f1
    references:                      Eyring, V., Bony, S., Meehl, G. A., Seni...
    source_id:                

In [10]:
import datatree

dt = datatree.open_datatree(
    's3://carbonplan-data-viewer/demo/ScenarioMIP.CCCma.CanESM5.ssp245.r1i1p1f1.day.GARD-SV.pyramid.tasmax.zarr',
    engine='zarr',
    chunks={},
)
dt

DataTree('None', parent=None)
│   Dimensions:  ()
│   Data variables:
│       *empty*
│   Attributes:
│       multiscales:  [{'datasets': [{'path': '0', 'pixels_per_tile': 128}, {'pat...
│       title:        multiscale data pyramid
│       version:      0.0.7.post1+gad0f0f4
├── DataTree('0')
│       Dimensions:  (time: 300, y: 128, x: 128)
│       Coordinates:
│         * time     (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-10-27
│         * x        (x) float32 -178.6 -175.8 -173.0 -170.2 ... 170.2 173.0 175.8 178.6
│         * y        (y) float32 89.3 87.89 86.48 85.08 ... -85.08 -86.48 -87.89 -89.3
│       Data variables:
│           tasmax   (time, y, x) float32 dask.array<chunksize=(30, 128, 128), meta=np.ndarray>
│       Attributes: (12/18)
│           Conventions:                     CF-1.8
│           activity_id:                     ScenarioMIP
│           cmip6_downscaling_contact:       hello@carbonplan.org
│           cmip6_downscaling_explainer:     https://carbonplan.org/research/cmip6-do...
│           cmip6_downscaling_institution:   CarbonPlan
│           cmip6_downscaling_license:       CC-BY-4.0
│           ...                              ...
│           member_id:                       r1i1p1f1
│           references:                      Eyring, V., Bony, S., Meehl, G. A., Seni...
│           regrid_method:                   bilinear
│           source_id:                       CanESM5
│           timescale:                       day
│           variable_id:                     tasmax
├── DataTree('1')
│       Dimensions:  (time: 300, y: 256, x: 256)
│       Coordinates:
│         * time     (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-10-27
│         * x        (x) float32 -179.3 -177.9 -176.5 -175.1 ... 175.1 176.5 177.9 179.3
│         * y        (y) float32 89.65 88.95 88.24 87.54 ... -87.54 -88.24 -88.95 -89.65
│       Data variables:
│           tasmax   (time, y, x) float32 dask.array<chunksize=(30, 128, 128), meta=np.ndarray>
│       Attributes: (12/18)
│           Conventions:                     CF-1.8
│           activity_id:                     ScenarioMIP
│           cmip6_downscaling_contact:       hello@carbonplan.org
│           cmip6_downscaling_explainer:     https://carbonplan.org/research/cmip6-do...
│           cmip6_downscaling_institution:   CarbonPlan
│           cmip6_downscaling_license:       CC-BY-4.0
│           ...                              ...
│           member_id:                       r1i1p1f1
│           references:                      Eyring, V., Bony, S., Meehl, G. A., Seni...
│           regrid_method:                   bilinear
│           source_id:                       CanESM5
│           timescale:                       day
│           variable_id:                     tasmax
└── DataTree('2')
        Dimensions:  (time: 300, y: 512, x: 512)
        Coordinates:
          * time     (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-10-27
          * x        (x) float32 -179.6 -178.9 -178.2 -177.5 ... 177.5 178.2 178.9 179.6
          * y        (y) float32 89.82 89.47 89.12 88.77 ... -88.77 -89.12 -89.47 -89.82
        Data variables:
            tasmax   (time, y, x) float32 dask.array<chunksize=(30, 128, 128), meta=np.ndarray>
        Attributes: (12/18)
            Conventions:                     CF-1.8
            activity_id:                     ScenarioMIP
            cmip6_downscaling_contact:       hello@carbonplan.org
            cmip6_downscaling_explainer:     https://carbonplan.org/research/cmip6-do...
            cmip6_downscaling_institution:   CarbonPlan
            cmip6_downscaling_license:       CC-BY-4.0
            ...                              ...
            member_id:                       r1i1p1f1
            references:                      Eyring, V., Bony, S., Meehl, G. A., Seni...
            regrid_method:                   bilinear
            source_id:                       CanESM5
            timescal

In [8]:
import zarr

In [9]:
zarr.consolidate_metadata(
    's3://carbonplan-data-viewer/demo/ScenarioMIP.CCCma.CanESM5.ssp245.r1i1p1f1.day.GARD-SV.pyramid.tasmax.zarr'
)

<zarr.hierarchy.Group '/'>